<a href="https://colab.research.google.com/github/almsalahuddin/Data-Processing-and-Visualization/blob/main/data_viz_bike_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quiz 6 # : CPSMA 4313 [Dr. Jacob's](https://github.com/nurfnick) Data processing visualization

## SQL Joins

### A L M SALAHUDDIN

## Learning Some SQL with BigQuery

First I will try to authenticate in o0rder to use the dataset from goodle cloud platform.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Above, it shows I authenticated. Now, I am able to use the dataset from BigQuery interface.

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT 
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_trips
0,1424786


The dataset I am using has total of 1424786 trips.

# Compute the average time for a trip based on starting points

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT 
  start_station_name,
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name

,start_station_name,Average_time_for_trip
0,Zilker Park West,27.168259
1,Toomey Rd @ South Lamar,28.115798
2,State Capitol @ 14th & Colorado,30.421842
3,Waller & 6th St.,23.265843
4,Pease Park,28.648889
...,...,...
188,Republic Square @ Guadalupe & 4th St.,23.067313
189,3rd & West,19.937466
190,3rd/West,34.152547
191,Nueces & 26th,16.805353


 # Compute how many trips start at each starting point

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT 
  start_station_name,
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,start_station_name,total_trips
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1


In [ ]:
%%bigquery --project data-viz-bike-share
SELECT 
  AVG(duration_minutes)
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,f0_
0,30.870428


Above, I calculated the average time for a trip.

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT duration_minutes, duration_minutes 
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` 
LIMIT 1000

,duration_minutes,duration_minutes_1
0,39,39
1,31,31
2,31,31
3,30,30
4,19,19
...,...,...
995,33,33
996,32,32
997,0,0
998,1,1


# second most popular starting station

Next, I will try to find out the second most popular trip station.

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT start_station_name, COUNT(*) as second_most_pop_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
HAVING second_most_pop_trip!= (SELECT MAX(second_most_pop_trip)as max_trip
FROM (
SELECT COUNT(*) as second_most_pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY second_most_pop_trip DESC
LIMIT 1

,start_station_name,second_most_pop_trip
0,Riverside @ S. Lamar,40635


Above, we can see Riverside @ S. Lamar station made the second most trip which is 40,635 times.


# Number of round trip

Now, I will calculate the number of round trips ecah statrting station made.

In [ ]:
%%bigquery --project data-viz-bike-share
SELECT start_station_name, SUM(round_trips) as number_of_round_trips
FROM (
  SELECT *, (CASE WHEN start_station_name = end_station_name THEN 1 ELSE 0 END) as round_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
)
GROUP BY start_station_name,end_station_name
ORDER BY number_of_round_trips DESC
LIMIT 200

,start_station_name,number_of_round_trips
0,21st & Speedway @PCL,13109
1,Riverside @ S. Lamar,12302
2,Zilker Park,9754
3,Rainey St @ Cummings,8676
4,2nd & Congress,5830
...,...,...
195,East 11th St. at Victory Grill,0
196,East 2nd & Pedernales,0
197,East 11th St. at Victory Grill,0
198,Capital Metro HQ - East 5th at Broadway,0


# Number of trip in an hour

Next, I will calculate number of trip took over an hour for duration in minutes for each starting station.

In [ ]:
%%bigquery --project data-viz-bike-share

SELECT *, COUNTIF(duration_minutes>60) as number_of_trip_took_over_an_hour
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY number_of_trip_took_over_an_hour DESC

,start_station_name,duration_minutes,number_of_trip_took_over_an_hour
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0


# Average time on trips based on starting station type

In [6]:
%%bigquery --project data-viz-bike-share

WITH t as(
SELECT name, property_type
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
)
SELECT t.property_type as start_station_type, 
    AVG (trips.duration_minutes) as average_min,
    COUNT(*) as number_of_trips,
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips
  JOIN t
  ON trips.start_station_name = t.name

GROUP BY t.property_type
ORDER by average_min

,start_station_type,average_min,number_of_trips
0,undetermined_parking,23.098028,225333
1,None,25.697646,68046
2,paid_parking,25.852535,280405
3,sidewalk,29.263569,432392
4,,33.502832,19776
5,parkland,39.375957,141806
6,nonmetered_parking,49.046326,11052


Above, We can see that trips starting from 'parkland' took longer than from 'sidewalk'.